In [1]:
import pandas as pd
import numpy as np

In [2]:
distances = pd.read_csv(
    '/Applications/University/Semester-7/Transportation Planning/Project/Phase II/average_distance.csv')
prices = pd.read_csv(
    '/Applications/University/Semester-7/Transportation Planning/Project/Phase II/average_price.csv')
demands = pd.read_csv(
    '/Applications/University/Semester-7/Transportation Planning/Project/Phase II/od_matrix.csv')
tip_payed_or_not = pd.read_csv(
    '/Applications/University/Semester-7/Transportation Planning/Project/Phase II/tip_paid.csv')

In [3]:
prices["average_price"].replace(-np.inf, -1000000, inplace=True)

/var/folders/2v/3vn5wnts369_zklm9mbmj8680000gn/T/ipykernel_52894/3613075562.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prices["average_price"].replace(-np.inf, -1000000, inplace=True)


In [4]:
distances.drop_duplicates(
    subset=['PULocationID', 'DOLocationID'], inplace=True)
distances.drop(columns=["time_group"], inplace=True)
distances.set_index(['PULocationID', 'DOLocationID'], inplace=True)
distances.head()

distance
PULocationID DOLocationID          
7            7             0.975790
             33            8.735833
             41            6.252105
             42            5.388462
             43            4.797222

In [5]:
prices.head()

,PULocationID,DOLocationID,time_group,average_price
0,7,7,Day,10.598080
1,7,7,Midnight,11.046981
2,7,7,Night,9.121099
3,7,33,Day,38.340476
4,7,33,Midnight,31.800000


In [6]:
demands[(demands["PULocationID"] == 116) & (demands["DOLocationID"] == 116)]

,PULocationID,DOLocationID,time_group,od
1014,116,116,Day,385.727668
1015,116,116,Night,46.061051
1016,116,116,Midnight,21.090923


In [7]:
tip_payed_or_not.head()

,PULocationID,DOLocationID,time_group,tip_paid
0,7,7,Day,0.274000
1,7,7,Midnight,0.358491
2,7,7,Night,0.494505
3,7,33,Day,0.809524
4,7,33,Midnight,0.000000


In [8]:
distances

distance
PULocationID DOLocationID           
7            7              0.975790
             33             8.735833
             41             6.252105
             42             5.388462
             43             4.797222
...                              ...
263          236           14.556667
             238            0.816939
             242            1.637931
             244            7.335000
             263            5.762333

[625 rows x 1 columns]

In [9]:
print(distances.shape, prices.shape, demands.shape, tip_payed_or_not.shape)

(625, 1) (1875, 4) (1875, 4) (1875, 4)


In [10]:
zones = demands["PULocationID"].unique()
times = demands["time_group"].unique()
zones, times

(array([  7,  33,  41,  42,  43,  61,  65,  74,  75,  76,  82,  95,  97,
        116, 129, 130, 166, 181, 188, 197, 236, 238, 242, 244, 263]),
 array(['Day', 'Night', 'Midnight'], dtype=object))

In [11]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus

x = LpVariable.dicts("x", (zones, zones, times), cat='Continuous', lowBound=0)
prob = LpProblem("Transportation Project Phase II", LpMaximize)

K = 1000
C = 40
pi = 0.9734
r = 5

prob += (
    sum(x[i][j][t] * distances.loc[(i, j), 'distance']
        for i in zones for j in zones for t in times) <= K
), "Constraint 1"


for i in zones:
    for j in zones:
        for t in times:
            d_ijt = demands.loc[
                (demands['PULocationID'] == i) &
                (demands['DOLocationID'] == j) &
                (demands['time_group'] == t), 'od'
            ].values[0]
            prob += x[i][j][t] <= d_ijt, f"MaxFlow_{i}_{j}_{t}"
prob += (
    sum(
        x[i][j][t] * (
            C +
            pi * r * tip_payed_or_not.loc[
                (tip_payed_or_not['PULocationID'] == i) &
                (tip_payed_or_not['DOLocationID'] == j) &
                (tip_payed_or_not['time_group'] == t), 'tip_paid'].values[0] +
            prices.loc[
                (prices['PULocationID'] == i) &
                (prices['DOLocationID'] == j) &
                (prices['time_group'] == t), 'average_price'].values[0]
        )
        for i in zones for j in zones for t in times
    )
), "Objective_function"


prob.solve()

/Users/sobhan/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sobhan/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/2v/3vn5wnts369_zklm9mbmj8680000gn/T/eb1c215c95534bbcb1fd42be1fd43c8d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/2v/3vn5wnts369_zklm9mbmj8680000gn/T/eb1c215c95534bbcb1fd42be1fd43c8d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1881 COLUMNS
At line 7507 RHS
At line 9384 BOUNDS
At line 9385 ENDATA
Problem MODEL has 1876 rows, 1875 columns and 3750 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-1875) rows, 37 (-1838) columns and 37 (-3713) elements
0  Obj -0 Primal inf 1000 (1) Dual inf 3015.7004 (37)
1  Obj 71835.974
Optimal - objective value 71835.974
After Postsolve, objective 71835.974, infeasibilities - dual 53.807286 (5), primal 0 (0)
Presolved model was optimal, ful

1

In [12]:
print("Objective Value:", prob.objective.value())

Objective Value: 72872.25527896904


In [13]:
for v in prob.variables():
    if v.varValue > 0:
        print(f"{v.name} = {v.varValue}")

x_116_129_Day = 10.265287
x_116_181_Night = 0.29581879
x_130_166_Night = 0.70871156
x_166_181_Day = 13.917572
x_166_181_Midnight = 0.76098884
x_166_181_Night = 1.6619446
x_181_116_Night = 0.12914739
x_181_188_Day = 153.1019
x_188_197_Day = 29.337399
x_188_197_Midnight = 1.6041184
x_188_197_Night = 3.5032785
x_236_116_Day = 16.147716
x_236_238_Day = 149.30447
x_236_238_Midnight = 8.1637109
x_236_238_Night = 17.828954
x_236_76_Day = 0.77988489
x_238_242_Day = 0.61028457
x_238_7_Day = 2.995004
x_244_129_Day = 55.581099
x_244_129_Night = 6.6371279
x_244_263_Day = 192.73217
x_244_263_Night = 23.014804
x_263_238_Day = 72.349713
x_263_76_Day = 1.8439026
x_263_76_Night = 0.22018667
x_65_65_Midnight = 13.084601
x_75_76_Day = 33.703268
x_75_76_Night = 4.0246217
x_76_7_Day = 39.522203
x_95_97_Day = 44.93156
x_95_97_Night = 5.3654302
x_97_116_Day = 6.7167166
x_97_41_Day = 34.19843
x_97_41_Midnight = 1.8699111
x_97_41_Night = 4.0837507
x_97_74_Day = 47.254251
x_97_74_Midnight = 2.5837809


In [14]:
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

x_116_116_Day = 0.0
x_116_116_Midnight = 0.0
x_116_116_Night = 0.0
x_116_129_Day = 10.265287
x_116_129_Midnight = 0.0
x_116_129_Night = 0.0
x_116_130_Day = 0.0
x_116_130_Midnight = 0.0
x_116_130_Night = 0.0
x_116_166_Day = 0.0
x_116_166_Midnight = 0.0
x_116_166_Night = 0.0
x_116_181_Day = 0.0
x_116_181_Midnight = 0.0
x_116_181_Night = 0.29581879
x_116_188_Day = 0.0
x_116_188_Midnight = 0.0
x_116_188_Night = 0.0
x_116_197_Day = 0.0
x_116_197_Midnight = 0.0
x_116_197_Night = 0.0
x_116_236_Day = 0.0
x_116_236_Midnight = 0.0
x_116_236_Night = 0.0
x_116_238_Day = 0.0
x_116_238_Midnight = 0.0
x_116_238_Night = 0.0
x_116_242_Day = 0.0
x_116_242_Midnight = 0.0
x_116_242_Night = 0.0
x_116_244_Day = 0.0
x_116_244_Midnight = 0.0
x_116_244_Night = 0.0
x_116_263_Day = 0.0
x_116_263_Midnight = 0.0
x_116_263_Night = 0.0
x_116_33_Day = 0.0
x_116_33_Midnight = 0.0
x_116_33_Night = 0.0
x_116_41_Day = 0.0
x_116_41_Midnight = 0.0
x_116_41_Night = 0.0
x_116_42_Day = 0.0
x_116_42_Midnight = 0.0
x_116_42_Nig

In [15]:
status = LpStatus[prob.status]

with open("modelresults.txt", "w") as f:
    f.write(f"Solver Status: {status}\n")
    f.write(f"Objective Value: {prob.objective.value()}\n")

    f.write("Non-zero variables:\n")
    for v in prob.variables():
        f.write(f"{v.name} = {v.varValue}\n")

In [16]:
prob.writeLP("model.lp")

[x_116_116_Day,
 x_116_116_Midnight,
 x_116_116_Night,
 x_116_129_Day,
 x_116_129_Midnight,
 x_116_129_Night,
 x_116_130_Day,
 x_116_130_Midnight,
 x_116_130_Night,
 x_116_166_Day,
 x_116_166_Midnight,
 x_116_166_Night,
 x_116_181_Day,
 x_116_181_Midnight,
 x_116_181_Night,
 x_116_188_Day,
 x_116_188_Midnight,
 x_116_188_Night,
 x_116_197_Day,
 x_116_197_Midnight,
 x_116_197_Night,
 x_116_236_Day,
 x_116_236_Midnight,
 x_116_236_Night,
 x_116_238_Day,
 x_116_238_Midnight,
 x_116_238_Night,
 x_116_242_Day,
 x_116_242_Midnight,
 x_116_242_Night,
 x_116_244_Day,
 x_116_244_Midnight,
 x_116_244_Night,
 x_116_263_Day,
 x_116_263_Midnight,
 x_116_263_Night,
 x_116_33_Day,
 x_116_33_Midnight,
 x_116_33_Night,
 x_116_41_Day,
 x_116_41_Midnight,
 x_116_41_Night,
 x_116_42_Day,
 x_116_42_Midnight,
 x_116_42_Night,
 x_116_43_Day,
 x_116_43_Midnight,
 x_116_43_Night,
 x_116_61_Day,
 x_116_61_Midnight,
 x_116_61_Night,
 x_116_65_Day,
 x_116_65_Midnight,
 x_116_65_Night,
 x_116_74_Day,
 x_116_74_Mid